<a href="https://colab.research.google.com/github/VitorFRodrigues/Polo_Tech_Americanas/blob/main/Polo_Tech/Modulo_05/Projeto/Projeto_Mod05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>